In [15]:
import pandas as pd
# Helpful for showing indexing information
pd.set_option('display.max_colwidth', 150)

import os
import pyterrier as pt

import platform
platform.uname()

uname_result(system='Linux', node='gl1016.arc-ts.umich.edu', release='4.18.0-305.45.1.el8_4.x86_64', version='#1 SMP Wed Apr 6 13:48:37 EDT 2022', machine='x86_64')

In [16]:
if not pt.started():
    pt.init()

In [17]:
df_anime=pd.read_csv('data_processed/anime_with_review.csv')
df_anime=df_anime.rename(columns={'uid':'docno'})
df_anime.head(1)

,docno,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link,wiki_link,anime_name,wiki_intro,review
0,28891,Haikyuu!! Second Season,"Following their participation at the Inter-High, the Karasuno High School volleyball team attempts to refocus their efforts, aiming to conquer the...","['Comedy', 'Sports', 'Drama', 'School', 'Shounen']","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/76662.jpg,https://myanimelist.net/anime/28891/Haikyuu_Second_Season,http://en.wikipedia.org/wiki/Haiky%C5%AB!!,Haikyū!!,"Junior high school student, Shoyo Hinata, becomes obsessed with volleyball after catching a glimpse of Karasuno High School playing in Nationals o...","Overall: 10 \r\n \r\nPlot: 10 \r\n \r\nHaikyuu! is a sports genre anime, mainly tackling about Volleyball. In this kind of story, there's only one..."


In [6]:
df_anime['docno']=df_anime['docno'].astype(str)
df_anime['popularity']=df_anime['popularity'].astype(str)
df_anime['ranked']=df_anime['ranked'].astype(str)
df_anime['score']=df_anime['score'].astype(str)
df_anime['episodes']=df_anime['episodes'].astype(str)

In [18]:

pt_index_path = './pt_index'

if not os.path.exists(pt_index_path + "/data.properties"):

    indexer = pt.DFIndexer(pt_index_path,overwrite=True)
    index_ref = indexer.index(
    df_anime['synopsis'],df_anime['docno'],df_anime['title'],df_anime['popularity'],df_anime['ranked'],df_anime['score'],
    df_anime['episodes'],df_anime['wiki_intro'],df_anime['review']
)

else:
    # if you already have the index, create an IndexRef from the data in pt_index_path
    # that we can use to load using the IndexFactory
    # TODO
    index_ref = pt.IndexRef.of(pt_index_path+"/data.properties")
index = pt.IndexFactory.of(index_ref)

In [19]:
bm25_150 = pt.BatchRetrieve(index, wmodel="BM25")%150
bm25=pt.BatchRetrieve(index,wmodel='BM25')
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")

In [20]:
query=pd.read_csv('data_processed/query_list.csv')
query.reset_index(inplace=True)
query.rename(columns={'Query':'query','index':'qid'},inplace=True)
train_query=query[query['Train/Test']=='Train'].drop(columns=['Train/Test'])
test_query=query[query['Train/Test']=='Test'].drop(columns=['Train/Test'])

In [21]:
df_query=train_query.append(test_query)
df_query=df_query.reset_index().drop(columns=['index'])

In [15]:
result=bm25_150(train_query)
result=result.merge(df_anime,on=['docno'],how='left')
result=result[['qid','docno','rank','score_x','query','title','synopsis','genre','wiki_intro','review']]

In [16]:
result.rename(columns={'score_x':'bm25_score'})
result.to_csv('system_rating/baseline_rating.csv',index=False)

In [17]:
result_test=bm25_150(test_query)
result_test=result_test.merge(df_anime,on=['docno'],how='left')
result_test=result_test[['qid','docno','rank','score_x','query','title','synopsis','genre','wiki_intro','review']]
result_test.rename(columns={'score_x':'bm25_score'})
result_test.to_csv('system_rating/baseline_rating_test.csv',index=False)

In [27]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
df_test=bm25("High school idol")


/home/zihuiliu/.local/lib/python3.9/site-packages/pyterrier/transformer.py:269: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  return self.transform(*args, **kwargs)


In [28]:
df_test

,qid,docid,docno,rank,score,query
0,1,2581,32526,0,15.116253,High school idol
1,1,1867,15051,1,14.667413,High school idol
2,1,2119,22189,2,13.289305,High school idol
3,1,229,24997,3,12.606818,High school idol
4,1,4499,10321,4,12.383107,High school idol
...,...,...,...,...,...,...
995,1,3407,3838,995,1.681246,High school idol
996,1,3832,1017,996,1.681246,High school idol
997,1,4398,395,997,1.681246,High school idol
998,1,672,17895,998,1.670446,High school idol


In [22]:
print(index.getCollectionStatistics().toString())

Number of documents: 4517
Number of terms: 20551
Number of postings: 214508
Number of fields: 0
Number of tokens: 255416
Field names: []
Positions:   false



## Evaluate baseline

### Naive random baseline

In [27]:
import random

def random_weighting(keyFreq, posting, entryStats, collStats):
    '''
    Computes the Pivoted Normalization score of this document for a given query
    
    :param keyFrequency(float): the weight of the term in the query, usually 1 except during PRF.
    :param posting(Posting): access to the information about the occurrence of the term in the current document (frequency, document length etc).
    :param entryStats(EntryStatistics): access to the information about the occurrence of the term in the whole index (document frequency, etc.).
    :param collStats(CollectionStatistics): access to the information about the index as a whole (number of documents, etc).
    :return: the Pivoted Normalization score of this document for the given query
    '''
    return random.random()
random_model=pt.BatchRetrieve(index,wmodel=random_weighting)

In [28]:


df_query_train=df_query[df_query['qid'].isin([0,1,2,3,4,5,6,7,8,9,20,21,22,23,24,25,26,27,28,29])]
df_query_train['qid']=df_query_train['qid'].astype('str')
df_label_train=pd.read_csv("labels.csv")
df_label_train['qid']=df_label_train['qid'].astype('str')
df_label_train['docno']=df_label_train['docno'].astype('str')
df_label_train['label']=df_label_train['label'].astype('int64')

df_query_test=df_query[df_query['qid'].isin([10,11,12,13,14,15,16,17,18,19,31,32,33,34,35,36,37,38,39])]
df_query_test['qid']=df_query_test['qid'].astype('str')
df_label_test=pd.read_csv("labels_test.csv")
df_label_test['qid']=df_label_test['qid'].astype('str')
df_label_test['docno']=df_label_test['docno'].astype('str')
df_label_test['label']=df_label_test['label'].astype('int64')

/tmp/ipykernel_2847140/1701384312.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query_train['qid']=df_query_train['qid'].astype('str')
/tmp/ipykernel_2847140/1701384312.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query_test['qid']=df_query_test['qid'].astype('str')


In [24]:
df_label_train[df_label_train['label'].isna()]

,qid,docno,label


In [33]:
from pyterrier.measures import *

# training set
pt.Experiment(
    [bm25,bm25_150,random_model,tf_idf],
    df_query_train,
    df_label_train,
    eval_metrics=["map", "ndcg",nDCG@5,nDCG@10,nDCG@15,nDCG@20,'P.10','mrt'])

,name,map,ndcg,nDCG@5,nDCG@10,nDCG@15,nDCG@20,P.10,mrt
0,BR(BM25),0.652690,0.791635,0.565184,0.507560,0.499254,0.494038,0.668421,697.998200
1,"RankCutoff(BR(BM25), 150)",0.652528,0.791121,0.565184,0.507560,0.499254,0.494038,0.668421,680.097786
2,BR(DPH),0.284922,0.630307,0.332835,0.330907,0.326697,0.318132,0.552632,882.800308
3,BR(TF_IDF),0.645893,0.789467,0.575318,0.511972,0.500193,0.492218,0.678947,688.461164


In [34]:
from pyterrier.measures import *

# test set
pt.Experiment(
    [bm25,bm25_150,random_model,tf_idf],
    df_query_test,
    df_label_test,
    eval_metrics=["map", "ndcg",nDCG@5,nDCG@10,nDCG@15,nDCG@20,'P.10','mrt'])

,name,map,ndcg,nDCG@5,nDCG@10,nDCG@15,nDCG@20,P.10,mrt
0,BR(BM25),0.651684,0.769736,0.470190,0.454131,0.463274,0.468618,0.716667,529.951314
1,"RankCutoff(BR(BM25), 150)",0.651464,0.768768,0.470190,0.454131,0.463274,0.468618,0.716667,529.816299
2,BR(DPH),0.334593,0.643975,0.396237,0.370382,0.355776,0.348157,0.594444,690.637470
3,BR(TF_IDF),0.647786,0.768069,0.462026,0.463406,0.463230,0.464580,0.722222,606.810072
